# IMPORTING LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
import tensorflow as tf


# IMPORTING DATASET

In [ ]:
dataset = pd.read_csv('train.csv')

In [ ]:
dataset.dtypes

In [ ]:
dataset.head()

In [ ]:
dataset.columns

# HANDLING MISSING VALUES 

In [ ]:
title_dataset = dataset.copy()
titles = []
import re
name = []
for i in range(dataset.shape[0]):
  name.append(dataset['Name'].values[i])
  name[i] = re.sub('[^a-zA-Z]', ' ', name[i])
  titles.append(name[i].split(' ')[2])
title_dataset['title'] = titles

In [ ]:
avg = title_dataset.groupby(by = 'title', axis = 0).mean()
avg

In [ ]:
avg_age = avg.loc[:, ['Age']]
avg_age

In [ ]:
avg_age.index

In [ ]:
title_list = ['', 'Capt', 'Castellana', 'Col', 'Don', 'Dr', 'Jonkheer', 'Major',
       'Manent', 'Master', 'Miss', 'Mlle', 'Mme', 'More', 'Mr', 'Mrs', 'Ms',
       'Rev', 'hoef', 'the']

In [ ]:
for i in range(title_dataset.shape[0]):
  if (pd.isnull(title_dataset['Age']).values[i]):
    for title in title_list:
      if (title_dataset['title'].values[i] == title):
        print(avg_age.loc[title, ['Age']].values)
        title_dataset['Age'].values[i] = avg_age.loc[title, ['Age']].values



In [ ]:
no_miss_data = title_dataset.loc[(pd.isnull(title_dataset['Embarked'])==0) & (pd.isnull(title_dataset['Age'])==0), :]

# DEPENDENT AND INDEPENDENT VARIABLE

In [ ]:
x = no_miss_data.loc[:, ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = no_miss_data.loc[:, ['Survived']]

# FEATURE SCALING

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = x.copy()
x_scaled.loc[:, ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']] = scaler.fit_transform(x.loc[:, ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']])

In [ ]:
print(x_scaled)

ONE HOT ENCODING AND LABEL ENCODING

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1, 6])], remainder = 'passthrough')
x_clean = ct.fit_transform(x_scaled)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_clean, y, test_size = 0.15, random_state = 42)

# LAYING OUT THE MODEL

In [ ]:
hidden_layer_size = 100
output_size = 1
model = tf.keras.Sequential([
                              tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                              tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                              tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                              tf.keras.layers.Dense(output_size, activation = 'sigmoid')
                            ])

COMPILING MODEL

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# TRAINING THE MODEL

In [ ]:
model.fit(x_train, y_train, validation_data = (x_validation, y_validation), batch_size = 32, epochs = 5, verbose = 2)

# TEST DATA ANALYSIS

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test

In [ ]:
pd.isnull(test).sum()

In [ ]:
test_title = test.copy()
titles_in_test = []
import re
name_test = []
for i in range(test.shape[0]):
  name_test.append(test['Name'].values[i])
  name_test[i] = re.sub('[^a-zA-Z]', ' ', name_test[i])
  titles_in_test.append(name_test[i].split(' ')[2])
test_title['title'] = titles_in_test

In [ ]:
for i in range(test_title.shape[0]):
  if (pd.isnull(test_title['Age']).values[i]):
    for title in title_list:
      if (test_title['title'].values[i] == title):
        print(avg_age.loc[title, ['Age']].values)
        test_title['Age'].values[i] = avg_age.loc[title, ['Age']].values



In [ ]:
pd.isnull(test_title).sum()

In [ ]:
fare_3 = test_title.loc[(test_title['Pclass'] == 3) & (test_title['title'] == 'Mr'), ['Fare', 'title']]


In [ ]:
sns.distplot(fare_3.loc[:, ['Fare']])

In [ ]:
missing_fare = fare_3['Fare'].mode().values


In [ ]:
missing_fare

In [ ]:
for i in range(test_title.shape[0]):
  if (pd.isnull(test_title['Fare']).values[i]):
    test_title['Fare'].values[i] = missing_fare[0]


TEST FEATURES 

In [ ]:
x_test = test_title.loc[:, ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

# TEST FEATURE SCALING

In [ ]:
test_scaler = StandardScaler()
x_test_scaled = x_test.copy()
x_test_scaled.loc[:, ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']] = test_scaler.fit_transform(x_test.loc[:, ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']])

# TEST FEATURE ENCODING

In [ ]:
x_test_cleaned = ct.transform(x_test_scaled)

# predicting outcomes

In [ ]:
pred_survive = (model.predict(x_test_cleaned) >= 0.5).astype(int)

In [ ]:
pred_survive

final formatting

In [ ]:
pass_id = np.arange(892, 1310)
submission = pd.DataFrame(np.concatenate((pass_id.reshape(-1,1), pred_survive.reshape(-1,1)), 1), columns = ['PassengerId', 'Survived'])


In [ ]:
submission

converting to csv file

In [ ]:
submission.to_csv('titanic_sub.csv', index = False, header = True)